In [1]:
! pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

  Using cached autogen_agentchat-0.4.7-py3-none-any.whl (66 kB)
  Using cached autogen_ext-0.4.7-py3-none-any.whl (161 kB)
  Using cached autogen_core-0.4.7-py3-none-any.whl (81 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl (9.4 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl (434 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl (64 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl (198 kB)
  Using cached azure_ai_inference-1.0.0b9-py3-none-any.whl (124 kB)
  Using cached azure_identity-1.20.0-py3-none-any.whl (188 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
  Using cached isodate-0.7.2-py3-none-any.whl (22 kB)
  Using cached cryptography-44.0.1-cp39-abi3-win_amd64.whl (3.2 MB)
  Using cached msal-1.31.1-py3-none-any.whl (113 kB)
  Using cached msal_extensions-1.2.0-py3-none-any.whl (19 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
  Using cached Deprecated-1.2.18-py2.p


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import httpx
import asyncio 
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.

config_list = [{
    "model": "deepseek-r1:14b",
    "base_url": "http://localhost:11434",
    "api_type": "ollama"
}]

# Configure the assistant
assistant_config = {
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

model_client = OpenAIChatCompletionClient(
    model="deepseek-r1:14b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
What is the weather in New York?


BadRequestError: Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:14b does not support tools', 'type': 'api_error', 'param': None, 'code': None}}

In [3]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="deepseek-r1:14b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": False,
        "json_output": False,
        "family": "unknown",
    },
)

response = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(response)

finish_reason='stop' content='<think>\n\n</think>\n\nThe capital of France is Paris.' usage=RequestUsage(prompt_tokens=10, completion_tokens=12) cached=False logprobs=None thought=None


In [2]:
! ollama --version

ollama version is 0.5.11
